<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-библиотек-и-данных" data-toc-modified-id="Загрузка-библиотек-и-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка библиотек и данных</a></span></li><li><span><a href="#Разделение-данных-на-обучающую-и-тестовую-выборки" data-toc-modified-id="Разделение-данных-на-обучающую-и-тестовую-выборки-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Разделение данных на обучающую и тестовую выборки</a></span></li><li><span><a href="#Очистка-и-лемматизация-текстовых-данных" data-toc-modified-id="Очистка-и-лемматизация-текстовых-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Очистка и лемматизация текстовых данных</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Обучение-модели-Логистической-регрессии" data-toc-modified-id="Обучение-модели-Логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучение модели Логистической регрессии</a></span></li><li><span><a href="#Обучение-модели-LGBMClassifier" data-toc-modified-id="Обучение-модели-LGBMClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение модели LGBMClassifier</a></span></li></ul></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span><ul class="toc-item"><li><span><a href="#Проверка-лучшей-модели-Логистической-регрессии-на-тестовой-выборке" data-toc-modified-id="Проверка-лучшей-модели-Логистической-регрессии-на-тестовой-выборке-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Проверка лучшей модели Логистической регрессии на тестовой выборке</a></span></li><li><span><a href="#Проверка-лучшей-модели-на-адекватность" data-toc-modified-id="Проверка-лучшей-модели-на-адекватность-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Проверка лучшей модели на адекватность</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Анализ текстовых комментариев  для интернет-магазина

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель проекта:**  
Обучить модель классифицировать комментарии на позитивные и негативные.   
В распоряжении имееется набор данных с разметкой о токсичности правок.  
В результате построения модели необходимо достичь значения метрики качества *F1* не меньше 0.75. 

**План выполнения проекта:**

1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Выводы по результатам работы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка данных

### Загрузка библиотек и данных

Загрузка библиотек

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

import re

from tqdm import notebook
from tqdm.notebook import tqdm
tqdm.pandas()


from sklearn.decomposition import TruncatedSVD

import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Определение глобальных констант

In [2]:
RANDOM_STATE = 12345

Загрузка данных

In [3]:
pth1 = 'toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2,)
else:
    print('Что-то пошло не так')
    
# убираем лишний столбец, дублирующий индекс.
df = df.drop('Unnamed: 0', axis=1)

#вывод параметров датасета для проверки    
display(df.shape)
display(df.info())
display(df.head())
display(df.describe())

(159292, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


###  Разделение данных на обучающую и тестовую выборки

In [4]:
features_train, features_test = train_test_split(df, stratify=df['toxic'], test_size=0.25, random_state=RANDOM_STATE)
target_train, target_test = features_train['toxic'], features_test['toxic']
features_train, features_test = features_train.drop('toxic', axis=1), features_test.drop('toxic', axis=1)

### Очистка и лемматизация текстовых данных

Определим  функцию для предобработки и токенизации текста на английском языке. 
Она удаляет стоп-слова, цифры и знаки пунктуации, переводит слова в нижний регистр, 
разбивает слова на токены, добавляет к ним POS-тег, лемматизирует с помощью WordNetLemmatizer.
Если при вызове функции был передан cache, то результаты лемматизации сохраняются в кэше, 
чтобы повторная обработка одного и того же слова не требовала лишних вычислений.
Возвращает функция список уникальных лемм

In [5]:
def preprocess_and_tokenize_text(text, cache=None):
    
    # если пришло что-то не то
    if not isinstance(text, str) or text == '':
        return []

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # удаление цифр, пунктуации и приведение к нижнему регистру
    text = text.translate(str.maketrans('', '', string.digits + string.punctuation)).lower()

    # токенизация
    tokens = word_tokenize(text)

    # Определяем соответствие между тегами частей речи NLTK и WordNet
    tag_map = {'N': wordnet.NOUN,
               'V': wordnet.VERB,
               'R': wordnet.ADV,
               'J': wordnet.ADJ}
    
    # лемматизация токенов с использованием кэша и учетом post_tag
    words_lem, added_words = [], set()
    for token in tokens:
        if len(token) > 1 and token not in stop_words:
            if cache is not None and token in cache:
                lem_token = cache[token]
            else:
                # Определение post_tag для слова
                post_tag = nltk.pos_tag([token])[0][1]
                # Приводим тег части речи к формату WordNet
                wn_tag = tag_map.get(post_tag[0].upper(), wordnet.NOUN)
                # Лемматизация слова с указанием post_tag
                lem_token = lemmatizer.lemmatize(token, wn_tag)
                if cache is not None:
                    cache[token] = lem_token
            # Проверяем, было ли уже добавлено такое слово
            if lem_token not in added_words:
                words_lem.append(lem_token)
                added_words.add(lem_token)

    return ' '.join(words_lem)

Применим эту функцию к нашим данным

In [6]:
%%time

cached_lemmas = {}

features_train['lemm_text'] = features_train['text'].progress_apply(lambda x: preprocess_and_tokenize_text(x, cached_lemmas))

features_test['lemm_text'] = features_test['text'].progress_apply(lambda x: preprocess_and_tokenize_text(x, cached_lemmas))

  0%|          | 0/119469 [00:00<?, ?it/s]

  0%|          | 0/39823 [00:00<?, ?it/s]

CPU times: total: 3min 46s
Wall time: 4min 12s


In [7]:
#проверка результата
display(features_train.head())

,text,lemm_text
129315,Accusation of vandalism? \n\nWhat was the caus...,accusation vandalism cause purpose make remove...
8868,Shorter Oxford English Dictionary\nArrived tod...,shorter oxford english dictionary arrive today...
75827,"Kim, stop! You are not helping here!!! This is...",kim stop help fan fire make hotter cooler
137701,My blocking by user Jayron32 \n\nI believe t...,block user jayron believe whilst warrant avoid...
28407,Simple quote\nDon't you think this essentially...,simple quote dont think essentially kantian ma...


Очистка от нелемматизированных данных

In [8]:
features_train = features_train.drop('text', axis=1)
features_test = features_test.drop('text', axis=1)

Создание корпусов слов

In [9]:
corpus_train = list(features_train['lemm_text'])
corpus_test = list(features_test['lemm_text'])

Обработка данных методом TF-IDF

In [10]:
vect = TfidfVectorizer()

features_train_tfidf = vect.fit_transform(corpus_train)
features_test_tfidf = vect.transform(corpus_test)

**Вывод:**  
В процессе подготовки данных было выполнено следующее:
- Загрузка данных
- Разбиение выборки на тренировочную и тестовую для обучения моделей.
- Очистка и лемматизация текстовых данных
- Создание корпусов слов
- Обработка данных методом TF-IDF

В результате, данные полностью готовы для обучения моделей

## Обучение моделей

Будем оценивать качество обучения на тренировочной выборке с применением кросс-валидации и подбором гиперпараметров для двух моделей, построенных на разных принципах: 
- модели Логистической регрессии - это линейная модель классификации
- модели LGBMClassifier - это модель классификации на основе деревьев с применением  градиентного бустинга

В результате будет выбрана лучшая модель, которая потом будет проверяться на тестовой выборке.

### Обучение модели Логистической регрессии

In [11]:
%%time

model_lr = LogisticRegression(random_state=RANDOM_STATE) 

# определение списка и диапазона гиперпараметров для подбора
grid_space = [{               
               'class_weight': [{0:1, 1:4}, {0:1, 1:6}, 'balanced', None]
             }]

# подбор параметров с кросс-валидацией
lr_grid = GridSearchCV(model_lr, 
                       param_grid=grid_space, 
                       cv=5, 
                       scoring='f1', 
                       n_jobs=-1)

model_grid_lr = lr_grid.fit(features_train_tfidf,target_train)

final_score_lr = model_grid_lr.best_score_

print('Гиперпараметры лучшей модели: '+str(model_grid_lr.best_params_))
print('Метрика RMSE лучшей модели: '+str(final_score_lr))

Гиперпараметры лучшей модели: {'class_weight': {0: 1, 1: 4}}
Метрика RMSE лучшей модели: 0.7824240674151615
CPU times: total: 2 s
Wall time: 39.6 s


C:\Users\user\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Обучение модели LGBMClassifier

In [12]:
%%time

model_lgbm = LGBMClassifier(random_state=RANDOM_STATE) 

# определение списка и диапазона гиперпараметров для подбора
grid_space = [{
                'learning_rate': [0.1, 0.01],
                'n_estimators': [50, 100, 150],
               }]

# подбор параметров с кросс-валидацией
lgbm_boost_grid = GridSearchCV(model_lgbm, 
                               param_grid=grid_space, 
                               cv=5, 
                               scoring='f1', 
                               n_jobs=-1)

model_grid_lgbm = lgbm_boost_grid.fit(features_train_tfidf,target_train)

final_score_lgbm = model_grid_lgbm.best_score_

print('Гиперпараметры лучшей модели: '+str(model_grid_lgbm.best_params_))
print('Метрика F1 лучшей модели: '+str(final_score_lgbm))

Гиперпараметры лучшей модели: {'learning_rate': 0.1, 'n_estimators': 150}
Метрика F1 лучшей модели: 0.7611583020053961
CPU times: total: 7min 40s
Wall time: 3min 21s


**Вывод**  
Обе модели при обучении показали результат выше необходимого порогового уровня F1 = 0.75  
Наилучший результат по метрике F1 = 0.78 показала модель Логистической регрессии со следующими гиперпараметрами:  
- class_weight = {0: 1, 1: 4}, 

    
Именно эта модель признана лучшей и будет выбрана для проверки на тестовой выборке.

## Тестирование лучшей модели

### Проверка лучшей модели Логистической регрессии на тестовой выборке

In [13]:
predictions_test = model_grid_lr.predict(features_test_tfidf)
print("Метрика F1 лучшей модели:", f1_score(target_test, predictions_test))

Метрика F1 лучшей модели: 0.7933316989458201


**Вывод:**  
Метрика F1 = 0.79 лучшей модели на тестовой выборке даже лучше, чем при обучении и заведомо выше целевого уровня 75, следовательно модель удвлетворяет требованиям заказчика.  

### Проверка лучшей модели на адекватность

Для прверки модели на адекватность сравним её с константной моделью классификации DummyClassifier, работающей по стратегии stratified. Такая модель предсказывает значения целевой переменной с учетом распределения классов в обучающем наборе данных и случайным образом выбирает классы с вероятностями, пропорциональными их частоте.

In [14]:
dummy_clf = DummyClassifier(strategy='stratified')

# обучение модели на тренировочной выборке
dummy_clf.fit(features_train_tfidf,target_train) 

# получение предсказания модели
predictions_dummy = dummy_clf.predict(features_test_tfidf) 

print("Модель Dummy, тестовая выборка")

# определение качества модели Dummy на тестовой выборке
print("F1:", f1_score(target_test, predictions_dummy)) 

Модель Dummy, тестовая выборка
F1: 0.09811413780352521


**Вывод:**    
Модель Dummy показала результат F1 = 0.10  
Это значительно хуже, чем результат нашей лучшей модели.  
Следовательно выбранная нами модель более, чем адекватна.

## Общий вывод

**В процессе выполнения проекта было выполнено следующее:**

- Загрузка данных
- Подготовка данных для обучения моделей:
 - Разбиение выборки на тренировочную и тестовую.
 - Очистка и лемматизация текстовых данных
 - Создание корпусов слов
 - Обработка данных методом TF-IDF
- Обучение на тренировочной выборке следующих моделей:
 - модели Логистической регрессии
 - модели градиентного бустинга LGBMClassifier
- Тестирование лучшей модели:
 - Оценка качества предсказания на тестовой выборке мо метрике F1
 - Проверка на адекватность сравнением с DummyClassifier
 
**Результат обучения моделей:**
 - Наивысшее качество по метрике F1 = 0.78 при обучении на тренировочной выборке показала модель Логистической регрессии с подобранными с использованием кросс-валидации гиперпараметрами, следовательно она и признана лучшей. 
 - Метрика F1 = 0.79 выбранной модели на тестовой выборке даже лучше, чем при обучении и заведомо выше целевого уровня 0.75, следовательно модель  Логистической регрессии удовлетворяет требованиям заказчика.  
 - Модель Логистической регрессии прошла проверку на адекватность, показав результат по метрике F1 значительно лучше, чем константная модель классификации DummyClassifier, работающей по стратегии stratified.
 
**Рекомендация сотрудникам интернет-магазина:**
- Предлагается использовать разработанную в данном проекте модель Логистической регрессии с подобранными гиперпараметрами для классификации комментариев на позитивные и негативные, в рамках нового сервиса, с целью поиска токсичных комментариев и отправки их на модерацию. 
- Данная модель показала наилучший результат по метрике F1 существенно выше требуемого значения.